In [40]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout, LSTM, Dense, Conv1D, MaxPooling1D, Flatten, LeakyReLU, Input, Concatenate
from tensorflow import keras
from keras import optimizers
import tensorflow as tf
import pickle
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, LeakyReLU

In [5]:
TRAIN_SET = './PTB_train_set.pickle'
TEST_SET = './PTB_test_set.pickle'

with open(TEST_SET, 'rb') as file:
    test_set = pickle.load(file)
    x_test = test_set['x']
    y_test = test_set['y']

with open(TRAIN_SET, 'rb') as file:
    train_set = pickle.load(file)
    x_train = train_set['x']
    y_train = train_set['y']
    
x_train = tf.expand_dims(x_train, axis=2)
x_test = tf.expand_dims(x_test, axis=2)
x_train.shape

TensorShape([13097, 187, 1])

In [46]:
model_CNN = Sequential()
model_CNN.add(Conv1D(5, kernel_size=3, strides=1, input_shape=x_train.shape[1:]))
model_CNN.add(LeakyReLU())
model_CNN.add(MaxPooling1D(pool_size=2, strides=2))
model_CNN.add(Conv1D(10, kernel_size=4, strides=1))
model_CNN.add(LeakyReLU())
model_CNN.add(MaxPooling1D(pool_size=2, strides=2))
model_CNN.add(Conv1D(20, kernel_size=4, strides=1))
model_CNN.add(LeakyReLU())
model_CNN.add(MaxPooling1D(pool_size=2, strides=2))
model_CNN.add(Flatten())
model_CNN.add(Dense(30))
model_CNN.add(LeakyReLU())
model_CNN.add(Dense(20))
model_CNN.add(LeakyReLU())
model_CNN.add(Dense(1, activation="softmax"))

opt = keras.optimizers.SGD(lr=0.003, momentum=0.7)
model_CNN.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

model_CNN.summary()
    

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, 185, 5)            20        
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 185, 5)            0         
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 92, 5)             0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 89, 10)            210       
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 89, 10)            0         
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 44, 10)            0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 41, 20)           

In [47]:
model_LSTM = Sequential()
model_LSTM.add(LSTM(64, input_shape=(187,1)))
model_LSTM.add(Dense(128, activation='relu'))
model_LSTM.add(Dropout(0.3))
model_LSTM.add(Dense(1, activation='softmax'))
opt = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model_CNN.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
model_LSTM.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 64)                16896     
_________________________________________________________________
dense_14 (Dense)             (None, 128)               8320      
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 129       
Total params: 25,345
Trainable params: 25,345
Non-trainable params: 0
_________________________________________________________________


In [33]:
#Demonstration of Ensemble Averaging
def EnsembleAveraging(values):
    factor = 1/len(values)
    total = 0
    for value in values:
        total += values[0]*factor
    return total

a = model_CNN(tf.expand_dims(x_train[0], axis=0))
b = model_LSTM(tf.expand_dims(x_train[0], axis=0))
print(a)
print(b)
EnsembleAveraging([a, b])

tf.Tensor([[0.51476103 0.48523897]], shape=(1, 2), dtype=float32)
tf.Tensor([[0.5 0.5]], shape=(1, 2), dtype=float32)


<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0.51476103, 0.48523897]], dtype=float32)>

In [54]:
#Demonstration of Dense Layer Fusion
a = model_CNN(tf.expand_dims(x_train[0], axis=0))
b = model_LSTM(tf.expand_dims(x_train[0], axis=0))

ab = Concatenate(axis=1)([a, b])

Dense(2, activation="softmax")(ab)

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0.62021255, 0.37978742]], dtype=float32)>

In [50]:
def EnsembleAveraging(values):
    factor = 1/len(values)
    total = 0
    for value in values:
        total += values[0]*factor
    return total
    

def build_model():
    inputs = Input(shape=(187, 1))
    
    x1 = model_LSTM(inputs)
    x2 = model_CNN(inputs)


    #outputs = EnsembleAveraging([x1, x2]) - was not able to train using this method
    
    x = Concatenate(axis=1)([x1, x2])
    outputs = Dense(1, activation="softmax")(x)
    
    # Compile
    model = tf.keras.Model(inputs, outputs, name="Ensemble")
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
    model.compile(
        optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"]
    )
    return model

ensemble = build_model()

In [52]:

history = ensemble.fit(x_train, y_train, batch_size=128, epochs=1, validation_data=(x_test, y_test), verbose=1)

103/103 [==============================] - 13s 130ms/step - loss: 0.5885 - accuracy: 0.7247 - val_loss: 0.6155 - val_accuracy: 0.6969
